## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Along,IN,28.1667,94.7667,73.40,83,91,1.83,overcast clouds
1,1,Tabory,RU,58.5198,64.5495,43.83,77,100,6.78,overcast clouds
2,2,Rikitea,PF,-23.1203,-134.9692,75.06,79,12,10.11,light rain
3,3,Avera,US,33.1940,-82.5271,80.47,50,1,16.49,clear sky
4,4,Kathmandu,NP,27.7167,85.3167,66.42,100,75,3.44,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Along,IN,28.1667,94.7667,73.40,83,91,1.83,overcast clouds
2,2,Rikitea,PF,-23.1203,-134.9692,75.06,79,12,10.11,light rain
5,5,Atuona,PF,-9.8000,-139.0333,78.12,77,36,18.34,scattered clouds
13,13,San Quintin,MX,30.4833,-115.9500,76.68,32,0,8.88,clear sky
15,15,Lagoa,PT,39.0500,-27.9833,70.68,92,100,27.74,overcast clouds
20,20,Bambous Virieux,MU,-20.3428,57.7575,73.22,78,40,12.66,scattered clouds
31,31,Saint-Philippe,RE,-21.3585,55.7679,72.95,75,40,16.35,scattered clouds
36,36,Souillac,MU,-20.5167,57.5167,73.36,78,40,12.66,scattered clouds
60,60,Najran,SA,17.4924,44.1277,70.45,21,12,1.12,few clouds
61,61,Ribeira Grande,PT,38.5167,-28.7000,71.91,86,100,36.60,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        169
City           169
Country        169
Lat            169
Lng            169
Max Temp       169
Humidity       169
Cloudiness     169
Wind Speed     169
Description    169
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Along,IN,73.40,overcast clouds,28.1667,94.7667,
2,Rikitea,PF,75.06,light rain,-23.1203,-134.9692,
5,Atuona,PF,78.12,scattered clouds,-9.8000,-139.0333,
13,San Quintin,MX,76.68,clear sky,30.4833,-115.9500,
15,Lagoa,PT,70.68,overcast clouds,39.0500,-27.9833,
20,Bambous Virieux,MU,73.22,scattered clouds,-20.3428,57.7575,
31,Saint-Philippe,RE,72.95,scattered clouds,-21.3585,55.7679,
36,Souillac,MU,73.36,scattered clouds,-20.5167,57.5167,
60,Najran,SA,70.45,few clouds,17.4924,44.1277,
61,Ribeira Grande,PT,71.91,overcast clouds,38.5167,-28.7000,


In [8]:
hotel_df.tail()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
686,Dawei,MM,74.37,light rain,14.0833,98.2000,
688,Tarakan,ID,74.70,broken clouds,3.3000,117.6333,
690,Monze,ZM,77.77,clear sky,-16.2833,27.4833,
694,Irbil,IQ,73.22,clear sky,36.1926,44.0106,
701,Bonthe,SL,79.02,broken clouds,7.5264,-12.5050,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():


    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions (used Places) API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [19]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [23]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [26]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [31]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

# 11b. Display the figure


Figure(layout=FigureLayout(height='420px'))